# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Aug 30 2022 1:51PM,245352,10,8463044,Eywa Pharma Inc.,Released
1,Aug 30 2022 1:50PM,245363,12,HH6683,Hush Hush,Released
2,Aug 30 2022 1:48PM,245362,10,MSP213536,"Methapharm, Inc.",Released
3,Aug 30 2022 1:48PM,245362,10,MSP213546,"Methapharm, Inc.",Released
4,Aug 30 2022 1:48PM,245362,10,MSP213549,"Methapharm, Inc.",Released
5,Aug 30 2022 1:48PM,245362,10,MSP213545,"Methapharm, Inc.",Released
6,Aug 30 2022 1:48PM,245362,10,MSP213551,"Methapharm, Inc.",Released
7,Aug 30 2022 1:48PM,245362,10,MSP213552,"Methapharm, Inc.",Released
8,Aug 30 2022 1:47PM,245361,10,MSP213543,"Methapharm, Inc.",Released
9,Aug 30 2022 1:47PM,245361,10,MSP213544,"Methapharm, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
48,245359,Released,1
49,245360,Released,7
50,245361,Released,5
51,245362,Released,6
52,245363,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
245359,NaN,NaN,1.0
245360,NaN,NaN,7.0
245361,NaN,NaN,5.0
245362,NaN,NaN,6.0
245363,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
245240,0.0,8.0,30.0
245259,4.0,57.0,3.0
245260,0.0,0.0,10.0
245276,0.0,0.0,1.0
245277,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
245240,0,8,30
245259,4,57,3
245260,0,0,10
245276,0,0,1
245277,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,245240,0,8,30
1,245259,4,57,3
2,245260,0,0,10
3,245276,0,0,1
4,245277,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,245240,,8,30
1,245259,4,57,3
2,245260,,,10
3,245276,,,1
4,245277,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Aug 30 2022 1:51PM,245352,10,Eywa Pharma Inc.
1,Aug 30 2022 1:50PM,245363,12,Hush Hush
2,Aug 30 2022 1:48PM,245362,10,"Methapharm, Inc."
8,Aug 30 2022 1:47PM,245361,10,"Methapharm, Inc."
13,Aug 30 2022 1:43PM,245360,15,"Virtus Pharmaceuticals, LLC"
20,Aug 30 2022 1:42PM,245340,15,"Chadwick Pharmaceuticals, Inc."
21,Aug 30 2022 1:40PM,245358,19,"AdvaGen Pharma, Ltd"
50,Aug 30 2022 1:39PM,245359,10,Yusen – 3D Matrix
51,Aug 30 2022 1:36PM,245357,19,"GCH Granules USA, Inc."
52,Aug 30 2022 1:33PM,245355,10,Yusen – 3D Matrix


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Aug 30 2022 1:51PM,245352,10,Eywa Pharma Inc.,,,1
1,Aug 30 2022 1:50PM,245363,12,Hush Hush,,,1
2,Aug 30 2022 1:48PM,245362,10,"Methapharm, Inc.",,,6
3,Aug 30 2022 1:47PM,245361,10,"Methapharm, Inc.",,,5
4,Aug 30 2022 1:43PM,245360,15,"Virtus Pharmaceuticals, LLC",,,7
5,Aug 30 2022 1:42PM,245340,15,"Chadwick Pharmaceuticals, Inc.",,,1
6,Aug 30 2022 1:40PM,245358,19,"AdvaGen Pharma, Ltd",,5,24
7,Aug 30 2022 1:39PM,245359,10,Yusen – 3D Matrix,,,1
8,Aug 30 2022 1:36PM,245357,19,"GCH Granules USA, Inc.",,,1
9,Aug 30 2022 1:33PM,245355,10,Yusen – 3D Matrix,,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 30 2022 1:51PM,245352,10,Eywa Pharma Inc.,1,,
1,Aug 30 2022 1:50PM,245363,12,Hush Hush,1,,
2,Aug 30 2022 1:48PM,245362,10,"Methapharm, Inc.",6,,
3,Aug 30 2022 1:47PM,245361,10,"Methapharm, Inc.",5,,
4,Aug 30 2022 1:43PM,245360,15,"Virtus Pharmaceuticals, LLC",7,,
5,Aug 30 2022 1:42PM,245340,15,"Chadwick Pharmaceuticals, Inc.",1,,
6,Aug 30 2022 1:40PM,245358,19,"AdvaGen Pharma, Ltd",24,5,
7,Aug 30 2022 1:39PM,245359,10,Yusen – 3D Matrix,1,,
8,Aug 30 2022 1:36PM,245357,19,"GCH Granules USA, Inc.",1,,
9,Aug 30 2022 1:33PM,245355,10,Yusen – 3D Matrix,1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 30 2022 1:51PM,245352,10,Eywa Pharma Inc.,1,,
1,Aug 30 2022 1:50PM,245363,12,Hush Hush,1,,
2,Aug 30 2022 1:48PM,245362,10,"Methapharm, Inc.",6,,
3,Aug 30 2022 1:47PM,245361,10,"Methapharm, Inc.",5,,
4,Aug 30 2022 1:43PM,245360,15,"Virtus Pharmaceuticals, LLC",7,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 30 2022 1:51PM,245352,10,Eywa Pharma Inc.,1.0,NaN,NaN
1,Aug 30 2022 1:50PM,245363,12,Hush Hush,1.0,NaN,NaN
2,Aug 30 2022 1:48PM,245362,10,"Methapharm, Inc.",6.0,NaN,NaN
3,Aug 30 2022 1:47PM,245361,10,"Methapharm, Inc.",5.0,NaN,NaN
4,Aug 30 2022 1:43PM,245360,15,"Virtus Pharmaceuticals, LLC",7.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 30 2022 1:51PM,245352,10,Eywa Pharma Inc.,1.0,0.0,0.0
1,Aug 30 2022 1:50PM,245363,12,Hush Hush,1.0,0.0,0.0
2,Aug 30 2022 1:48PM,245362,10,"Methapharm, Inc.",6.0,0.0,0.0
3,Aug 30 2022 1:47PM,245361,10,"Methapharm, Inc.",5.0,0.0,0.0
4,Aug 30 2022 1:43PM,245360,15,"Virtus Pharmaceuticals, LLC",7.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,3679848,55.0,34.0,8.0
12,245363,1.0,0.0,0.0
15,981295,23.0,89.0,6.0
19,736058,25.0,6.0,0.0
20,1226549,125.0,11.0,0.0
21,2452844,8.0,2.0,0.0
22,245347,1.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,3679848,55.0,34.0,8.0
1,12,245363,1.0,0.0,0.0
2,15,981295,23.0,89.0,6.0
3,19,736058,25.0,6.0,0.0
4,20,1226549,125.0,11.0,0.0
5,21,2452844,8.0,2.0,0.0
6,22,245347,1.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,55.0,34.0,8.0
1,12,1.0,0.0,0.0
2,15,23.0,89.0,6.0
3,19,25.0,6.0,0.0
4,20,125.0,11.0,0.0
5,21,8.0,2.0,0.0
6,22,1.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,55.0
1,12,Released,1.0
2,15,Released,23.0
3,19,Released,25.0
4,20,Released,125.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,19,20,21,22
Status,,,,,,,
Completed,8.0,0.0,6.0,0.0,0.0,0.0,0.0
Executing,34.0,0.0,89.0,6.0,11.0,2.0,0.0
Released,55.0,1.0,23.0,25.0,125.0,8.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,19,20,21,22
0,Completed,8.0,0.0,6.0,0.0,0.0,0.0,0.0
1,Executing,34.0,0.0,89.0,6.0,11.0,2.0,0.0
2,Released,55.0,1.0,23.0,25.0,125.0,8.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,19,20,21,22
0,Completed,8.0,0.0,6.0,0.0,0.0,0.0,0.0
1,Executing,34.0,0.0,89.0,6.0,11.0,2.0,0.0
2,Released,55.0,1.0,23.0,25.0,125.0,8.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()